# M3.3 Study - Effect of K-anonymity on Titanic unalive prediction 

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import anonypy
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 

In [2]:
titanic = sns.load_dataset('titanic')
titanicAnonymized = sns.load_dataset('titanic')

print(titanicAnonymized.columns)

# print(titanicAnonymized['embarked'].value_counts())

catagorical = set(('sex', 'embarked'))

for name in catagorical:
    titanicAnonymized[name] = titanicAnonymized[name].astype("category");

featureColumns = ['sex', 'age', 'sibsp', 'parch', 'fare'] # Removes: pclass, deck, 'class', 'who', 'adult_male', 'embark_town', 'alive', 'alone', 'embarked'
sensitiveColumn = "survived"

print(featureColumns + [sensitiveColumn])

p = anonypy.Preserver(titanicAnonymized, featureColumns, sensitiveColumn)
rows = p.anonymize_k_anonymity(k=10)

titanicAnonymized = pd.DataFrame(rows)

titanicAnonymized = titanicAnonymized.loc[titanicAnonymized.index.repeat(titanicAnonymized['count'])].reset_index(drop=True)
titanicAnonymized = titanicAnonymized.drop(columns=['count'])



def transform_cell(cell):
    # If not a list, return as-is
    if not isinstance(cell, list):
        return cell
    
    # Flatten the comma-separated string inside the list
    if len(cell) == 1 and isinstance(cell[0], str) and ',' in cell[0]:
        parts = cell[0].split(',')
    else:
        parts = cell

    # Remove any "nan" strings
    cleaned = [v for v in parts if v != "nan"]

    if len(cleaned) == 1:
        item = cleaned[0]
        # Handle range like '15.0-16.0'
        if '-' in item and not item.startswith('-') and any(char.isdigit() for char in item):
            try:
                lower, upper = map(float, item.split('-'))
                return [lower, upper]
            except ValueError:
                return item
        return item  # categorical

    elif len(cleaned) > 1:
        return cleaned

    return None  # If all were "nan", return None (or np.nan if preferred)



# Apply transformation to all cells except the last two columns (which are int)
cols_to_transform = titanicAnonymized.columns.difference(['survived', 'count'])
titanicAnonymized[cols_to_transform] = titanicAnonymized[cols_to_transform].map(transform_cell)


titanicAnonymized.head(10)



Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')
['sex', 'age', 'sibsp', 'parch', 'fare', 'survived']


,sex,age,sibsp,parch,fare,survived
0,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0
1,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0
2,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0
3,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0
4,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0
5,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0
6,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0
7,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0
8,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0
9,male,"[12.0, 28.0]","[1.0, 2.0]",0,"[11.2417, 108.9]",0


In [3]:
titanic = sns.load_dataset('titanic')
titanic.head()
X = titanic['sex']
X.tail()

886      male
887    female
888    female
889      male
890      male
Name: sex, dtype: object

In [8]:
# Step 1: Define X (features) and y (target)
# X = titanic.drop(columns=['survived'])  # 'survived' is the target column in the Seaborn Titanic dataset
X = titanic[['sex', 'age', 'sibsp', 'parch', 'fare']].copy()
y = titanic['survived']

# Step 2: Handle missing values and encode categorical variables
X['age'] = X['age'].fillna(X['age'].mean())  # Fill missing values in 'age' with the mean
X['fare'] = X['fare'].fillna(X['fare'].mean())  # Fill missing values in 'fare' with the mean

# Encode categorical variables
X = pd.get_dummies(X, columns=['sex'], drop_first=True)  # One-hot encode 'sex' and 'embarked'
# Step 3: Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Train a logistic regression model
model = LogisticRegression()
model.fit(x_train, y_train)

# Step 5: Evaluate the model
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)   
print(f'Accuracy: {accuracy}')

Accuracy: 0.8059701492537313


In [ ]:
# Step 1: Define X (features) and y (target)
# X = titanic.drop(columns=['survived'])  # 'survived' is the target column in the Seaborn Titanic dataset
X = titanicAnonymized.drop(columns=['survived']).copy()
print(X.columns)
y = titanicAnonymized['survived']

for col in X.columns:
    for i in range(len(X[col])):
        if not isinstance(X[col][i], list):
            continue
        X.loc[i, col] = (X[col][i][0]+X[col][i][1])/2

# Step 2: Handle missing values and encode categorical variables
# X['age'] = X['age'].fillna(X['age'].mean())  # Fill missing values in 'age' with the mean
# X['fare'] = X['fare'].fillna(X['fare'].mean())  # Fill missing values in 'fare' with the mean

# Encode categorical variables
X = pd.get_dummies(X, columns=['sex'], drop_first=True)  # One-hot encode 'sex' and 'embarked'
# Step 3: Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 4: Train a logistic regression model
model = LogisticRegression()
model.fit(x_train, y_train)

# Step 5: Evaluate the model
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Index(['sex', 'age', 'sibsp', 'parch', 'fare'], dtype='object')
Accuracy: 0.7798165137614679
